In [ ]:
import pandas as pd
import hvplot.pandas
from pathlib import Path
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing

In [ ]:
def minMaxScale(df):
    columns = ['High','Low','Volume']
    min_max_scaler = preprocessing.MinMaxScaler()
    normDf = df.copy()
    normDf[columns] = pd.DataFrame(min_max_scaler.fit_transform(normDf[columns]))
    return normDf

def readStock(filename):
    filePath = basePath+filename
    df = pd.read_csv(filePath).dropna()
    df['Date'] = pd.to_datetime(df['Date'])
    df1 = df[['Date','High','Low','Volume']]
    
    return minMaxScale(df1)

In [ ]:
ABBV_Df = readStock("ABBV.csv")
AMN_Df= readStock("AMN.csv")
DHC_Df= readStock("DHC.csv")
GILD_Df= readStock("GILD.csv")
GSK_Df= readStock("GSK.csv")
HCA_Df= readStock("HCA.csv")
JNJ_Df= readStock("JNJ.csv")
OHI_Df= readStock("OHI.csv")
SNY_Df = readStock("SNY.csv")
TAK_Df= readStock("TAK.csv")

In [ ]:
ABBV_Df.head(10)

In [ ]:
Healthcare_df = pd.concat([ABBV_Df,
AMN_Df,
DHC_Df,
GILD_Df,
GSK_Df,
HCA_Df,
JNJ_Df,
OHI_Df,
SNY_Df,
TAK_Df])

In [3]:
Healthcare_df.head(10)

NameError: name 'Healthcare_df' is not defined

In [ ]:
Healthcare_df.info()

In [ ]:
Healthcare_df.describe()

In [ ]:
Healthcare_df.head(10)

In [ ]:
df_Healthcare = Healthcare_df.groupby('Date')[['High','Low','Volume']].mean().dropna()

In [ ]:
df_Healthcare

In [ ]:
df_Healthcare.describe()

In [ ]:
fig, ax = plt.subplots(figsize=(16, 10))
ax.plot(df_Healthcare.Volume, color='grey', linestyle= '-')
ax.set(title = "Historical stock Volume trading",
       xlabel = "Date",
       ylabel = "Volume Trading")


plt.ylim(0, 0.7)

In [ ]:
df_Healthcare.to_csv('Scaled_Data_Averaged.csv', encoding='utf-8')

In [ ]:
Sorted_df = df_Healthcare.sort_values(by=['Date'])
Sorted_df

In [ ]:
fig, ax = plt.subplots(figsize=(16, 10))
ax.plot(Sorted_df.High, color='red', linestyle= '--')
ax.plot(Sorted_df.Low, color='green')
ax.set(title = "High & Low Prices of Automobile Companies",
       xlabel = "Date",
       ylabel = "Stock Prices")


plt.ylim(0, 1)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [ ]:
train_df = Sorted_df[(Sorted_df.index < '2023-01-01')]
X_train = train_df[['High','Low']]
y_train = train_df['Volume']


test_df = Sorted_df[(Sorted_df.index >= '2023-01-01')]
X_test = test_df[['High', 'Low']]
y_test = test_df['Volume']

  
print("TRAIN")
print("Y train Shape: " + str(y_train.shape))
print("X train Shape: " + str(X_train.shape))

print("\nTEST")
print("Y test Shape: " + str(y_test.shape))
print("X test Shape: " + str(X_test.shape))

In [ ]:
model = LinearRegression()

In [ ]:
model.fit(X_train, y_train)

In [ ]:
print('Intercept:', model.intercept_)
print('Coefficients:', model.coef_)

In [ ]:
r2 = model.score(X_train, y_train)
print("Model R2: " + str(r2))

In [ ]:
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms

In [ ]:
X = sm.add_constant(X_train) # adding a constant

olsmod = sm.OLS(y_train, X).fit()
print(olsmod.summary())

In [ ]:
print('R2 score:', olsmod.rsquared)

In [ ]:
print("\nTEST")
print("Y test Shape: " + str(y_test.shape))
print("X test Shape: " + str(X_test.shape))

In [ ]:
Predictions = model.predict(X_test)

print("prediction shape: " + str(Predictions.shape))

In [ ]:
Pred_df=pd.DataFrame( {"Prediction": Predictions, "Actual": y_test})
Sorted_pred_df =Pred_df.sort_values(by=['Date'])
Sorted_pred_df

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
ax.plot(Sorted_pred_df.Prediction, color='blue', linestyle= '--')
ax.plot(Sorted_pred_df.Actual, color='green')
ax.set(title = "High & Low Prices of Healthcare Companies",
       xlabel = "Date",
       ylabel = "Stock Prices")


plt.ylim(0, 0.2)

In [ ]:
fig, ax = plt.subplots(figsize=(16, 10))
ax.scatter(Sorted_pred_df.index,Sorted_pred_df.Prediction,color='blue', linestyle= '--')
ax.plot(Sorted_pred_df.Actual, color='green')
plt.ylim(0, 0.2)

In [ ]:
# Prepare data
X = df_Healthcare[['High', 'Low']].values.reshape(-1,2)
Y = df_Healthcare['Volume']

# Create range for each dimension
x = X[:, 0]
y = X[:, 1]
z = Y



xx_pred = np.linspace(0, 1,20)  # range of High  values

yy_pred = np.linspace(0,1,20)  # range of Low values
xx_pred, yy_pred = np.meshgrid(xx_pred, yy_pred)
model_viz = np.array([xx_pred.flatten(), yy_pred.flatten()]).T

In [ ]:
predicted = model.predict(model_viz)

# Evaluate model by using it's R^2 score 
r2 = model.score(X, Y)

# Plot model visualization
plt.style.use('fivethirtyeight')

fig = plt.figure(figsize=(12, 6))

ax1 = fig.add_subplot(131, projection='3d')
ax2 = fig.add_subplot(132, projection='3d')
ax3 = fig.add_subplot(133, projection='3d')

axes = [ax1, ax2, ax3]

for ax in axes:
    ax.plot(x, y, z, color='#32a891', zorder=15, linestyle='-', marker='*', alpha=0.5)
    ax.scatter(xx_pred.flatten(), yy_pred.flatten(), predicted, facecolor=(0,0,0,0), s=20, edgecolor='#150e57')
    ax.set_xlabel('High Price of Stocks', fontsize=12)
    ax.set_ylabel('Low Prices of Stocks', fontsize=12)
    ax.set_zlabel('Volume trading ', fontsize=12)
    ax.locator_params(nbins=4, axis='x')
    ax.locator_params(nbins=5, axis='x')

    


ax1.view_init(elev=20, azim=-60)
ax2.view_init(elev=25, azim=15)
ax3.view_init(elev=15, azim=60)

fig.suptitle('Multi-Linear Regression Model Visualization ($R^2 = %.2f$)' % r2, fontsize=15, color='k')

fig.tight_layout()

In [ ]:
import seaborn as sns

In [ ]:
corr = Healthcare_df[['Volume', 'High', 'Low']].corr()
print('Pearson correlation coefficient matrix of each variables:\n', corr)

# Generate a mask for the diagonal cell
mask = np.zeros_like(corr, dtype=np.bool)
#np.fill_diagonal(mask, val=True)

# Initialize matplotlib figure
fig, ax = plt.subplots(figsize=(4, 3))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True, sep=100)
cmap.set_bad('grey')

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmin=-1, vmax=1, center=0, linewidths=.5)
fig.suptitle('Pearson correlation coefficient matrix', fontsize=14)
ax.tick_params(axis='both', which='major', labelsize=10)
# fig.tight_layout()